数据来源于[京东JData算法大赛-高潜用户购买意向预测](https://www.datafountain.cn/competitions/247/datasets).通过Python随机取了10000个用户和500个商品作为项目数据。

In [9]:
import pandas as pd
import psycopg2.extras as pe
import sqlite3
from scipy.stats import norm
import random
import numpy as np
import faker
import psycopg2

In [10]:
project_dir = r'/mnt/data-ubuntu/Projects/user_profile'
seed = 123
fake = faker.Faker('zh_CN')
faker.Faker.seed(123)

In [11]:
import sys
sys.path.insert(0,project_dir + r'/src')

In [14]:
import sql_queries

In [13]:
import importlib
importlib.reload(sql_queries)

<module 'sql_queries' from '/mnt/data-ubuntu/Projects/user_profile/src/sql_queries.py'>

In [62]:
project_dir = r'/mnt/data-ubuntu/Projects/user_profile'
conn = psycopg2.connect(user = 'test',
                        password = '123456',
                        host = '127.0.0.1',
                        port = '5432',
                        database = "user_profile_dw")

In [74]:
conn.close()

In [63]:
s = ""
s += "SELECT"
s += " table_schema"
s += ", table_name"
s += " FROM information_schema.tables"
s += " WHERE"
s += " ("
s += " table_schema = 'public'"
s += " )"
s += "OR (TABLE_SCHEMA = 'dw')"
s += " ORDER BY table_schema, table_name;"
pd.read_sql(s, conn)

,table_schema,table_name
0,public,events
1,public,orders
2,public,products
3,public,time
4,public,users


In [56]:
sql = """
CREATE TABLE dw.test(
    line_id SERIAL PRIMARY KEY,
    qty INTEGER
);
"""
c = conn.cursor()
c.execute(sql)

sql = """
SELECT *
FROM DW.TEST
"""
pd.read_sql(sql, conn)

,line_id,qty


In [27]:
# Drop tables
c = conn.cursor()
for q in sql_queries.drop_table_queries:
    c.execute(q)
    conn.commit()
# Create tables
for q in sql_queries.create_table_queries:
    c.execute(q)
    conn.commit()

In [66]:
sql = """
SELECT 
*
FROM users
"""
df_users = pd.read_sql(sql, conn)


In [257]:
import glob
lst_files = glob.glob(project_dir + r'/input/ecommerce_cosmetics/*.csv')

In [258]:
lst_files

['/mnt/data-ubuntu/Projects/user_profile/input/ecommerce_cosmetics/2019-Dec.csv',
 '/mnt/data-ubuntu/Projects/user_profile/input/ecommerce_cosmetics/2019-Nov.csv',
 '/mnt/data-ubuntu/Projects/user_profile/input/ecommerce_cosmetics/2019-Oct.csv',
 '/mnt/data-ubuntu/Projects/user_profile/input/ecommerce_cosmetics/2020-Feb.csv',
 '/mnt/data-ubuntu/Projects/user_profile/input/ecommerce_cosmetics/2020-Jan.csv']

In [19]:
%%time
with open(project_dir + r'/input/ecommerce_cosmetics/2019-Nov.csv', 'r') as f:
    next(f) # Skip the header row.
    c.copy_from(f,
                'staging_events',
                sep=',',
                columns=['event_time',
                         'event_type',
                         'product_id',
                         'category_id',
                         'category_code',
                         'brand',
                         'price',
                         'user_id',
                         'user_session'])
    conn.commit()

CPU times: user 590 ms, sys: 157 ms, total: 747 ms
Wall time: 29.7 s


In [222]:
c = conn.cursor()
c.execute(sql_queries.events_table_insert)
conn.commit()

In [223]:
c = conn.cursor()
c.execute(sql_queries.products_table_insert)
conn.commit()

In [224]:
c = conn.cursor()
c.execute(sql_queries.orders_table_insert)
conn.commit()

In [225]:
c = conn.cursor()
c.execute(sql_queries.time_table_insert)
conn.commit()

In [226]:
sql = """
SELECT
    DISTINCT
    user_id
FROM events
"""
df_users = pd.read_sql(sql, conn)

In [64]:
df_users

NameError: name 'df_users' is not defined

In [228]:
def generate_gender():
    """
    Generate random gender. Assume 80% of the customers are female.
    """
    if random.random()<=0.2:
        return 'M'
    else:
        return 'F'

num_users = df_users.shape[0]

# Create df_users
df_users['user_name'] = [fake.user_name() for _ in range(num_users)]
df_users['name'] = [fake.name() for _ in range(num_users)]
df_users['gender'] = [generate_gender() for _ in range(num_users)]
df_users['mail'] = [fake.email() for _ in range(num_users)]
df_users['province'] = [fake.province() for _ in range(num_users)]
# The age of the users consists of two parts. 80% is created by norm
# and the remaining is created by random numbers between 20 and 60.
users_norm = int(num_users*.8)
df_users['age'] = np.append(np.round(norm.rvs(30, 4, users_norm)),
                                  [random.randint(20, 60) for _ in range(num_users - users_norm)])



In [234]:
c = conn.cursor()
pe.execute_batch(c, sql_queries.users_table_insert, df_users.values)
conn.commit()

In [239]:
c = conn.cursor()
c.execute(sql_queries.staging_events_table_drop)
conn.commit()

In [262]:
a = None
if not a:
    print(1)

1


In [240]:
sql = """
SELECT
    *
FROM STAGING_EVENTS
LIMIT 5000
"""
pd.read_sql(sql, conn)

DatabaseError: Execution failed on sql '
SELECT
    *
FROM STAGING_EVENTS
LIMIT 5000
': relation "staging_events" does not exist
LINE 4: FROM STAGING_EVENTS
             ^


In [460]:
# purchase directly
df_events.query('user_session == "93f5a7c3-1f82-6965-966b-c465921e1821"')

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
9769,2019-11-25 19:35:58 UTC,purchase,5879287,1487580008607122425,NaN,masura,5.54,574814206,93f5a7c3-1f82-6965-966b-c465921e1821
9770,2019-11-25 19:35:58 UTC,purchase,5879287,1487580008607122425,NaN,masura,5.54,574814206,93f5a7c3-1f82-6965-966b-c465921e1821


In [463]:
df_products = df_events.loc[:, ['product_id', 'category_id', 'category_code', 'brand']]
df_products.sort_values(['product_id','category_id','category_code','brand'], inplace=True)
df_products = df_products.groupby('product_id').nth(0).reset_index()

In [490]:
df_products

,product_id,category_id,category_code,brand
0,3809,1487580005343953762,NaN,entity
1,4431,1487580009387261981,NaN,cnd
2,4723,1487580006317032337,NaN,NaN
3,4758,1487580010796548183,NaN,runail
4,4781,1487580006317032337,NaN,NaN
...,...,...,...,...
1995,5908799,1487580008011531230,NaN,dewal
1996,5909012,1487580005427839846,NaN,irisk
1997,5909052,1487580005427839846,NaN,irisk
1998,5909238,2187790129827939246,NaN,dewal


In [470]:
df_orders = df_events[df_events.event_type=='purchase'].loc[:, ['user_session', 'event_time', 'product_id', 'user_id', 'price']]
df_orders['created_time'] = df_orders.loc[:, ['user_session', 'event_time']].sort_values(['user_session', 'event_time']).groupby('user_session')['event_time'].transform('nth', -1)
df_orders = df_orders.drop('event_time', axis = 1)

In [483]:
# The price could be different for the sample product in a user_session.
df_orders['qty'] = df_orders.groupby(['user_session', 'product_id', 'price'])['user_id'].transform('count')
df_orders = df_orders.drop_duplicates().reset_index(drop=True)

In [491]:
df_orders

,user_session,product_id,user_id,price,created_time,qty
0,3780e751-80bb-4892-aed7-c72daf2ef9c6,5844397,557466625,2.38,2019-11-01 18:25:14 UTC,1
1,b16fb822-8904-49f9-86fe-469a8e9bb84f,5844397,457987436,2.38,2019-11-01 20:48:26 UTC,1
2,bd7cf733-dd73-4d09-8df4-c16cc05d222c,5844397,552185190,2.38,2019-11-03 18:41:34 UTC,1
3,e638e59b-e3e6-5a96-c535-060a1b506438,5844397,543665202,2.38,2019-11-04 14:09:20 UTC,1
4,46d53300-e884-4098-be62-a422f9df2988,5844397,499356552,2.38,2019-11-05 17:39:27 UTC,1
...,...,...,...,...,...,...
13324,f58abddb-b147-4ea5-8e2f-5a6ff6e2f9f4,5908190,570033309,1.43,2019-11-29 14:23:17 UTC,1
13325,9f729b71-3877-9415-6570-6a0e81d99441,5908190,542768063,1.43,2019-11-29 16:01:27 UTC,1
13326,74d0f474-f9cd-4e36-932e-b3fcf73595a6,5856537,503269572,9.52,2019-11-28 07:06:15 UTC,1
13327,0d2fc105-b32e-4d75-90a1-ee44d1d0d39d,5909238,579137281,2.62,2019-11-30 17:49:06 UTC,1


In [489]:
df_events.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:09 UTC,cart,5844397,1487580006317032337,NaN,NaN,2.38,553329724,2067216c-31b5-455d-a1cc-af0575a34ffb
1,2019-11-01 07:29:50 UTC,remove_from_cart,5844397,1487580006317032337,NaN,NaN,2.38,553329724,69b84261-c592-4980-89ae-dcd7b761f51f
2,2019-11-01 12:48:19 UTC,view,5844397,1487580006317032337,NaN,NaN,2.38,566097223,41e58e78-ab43-4451-8b3a-415e74d3caf8
3,2019-11-01 14:30:16 UTC,view,5844397,1487580006317032337,NaN,NaN,2.38,566097223,41e58e78-ab43-4451-8b3a-415e74d3caf8
4,2019-11-01 16:50:43 UTC,view,5844397,1487580006317032337,NaN,NaN,2.38,476624689,533e0004-2213-4313-9809-1c2b9026dc57


In [487]:
# Generate user table


,user_session,product_id,user_id,price,created_time,qty
0,3780e751-80bb-4892-aed7-c72daf2ef9c6,5844397,557466625,2.38,2019-11-01 18:25:14 UTC,1
1,b16fb822-8904-49f9-86fe-469a8e9bb84f,5844397,457987436,2.38,2019-11-01 20:48:26 UTC,1
2,bd7cf733-dd73-4d09-8df4-c16cc05d222c,5844397,552185190,2.38,2019-11-03 18:41:34 UTC,1
3,e638e59b-e3e6-5a96-c535-060a1b506438,5844397,543665202,2.38,2019-11-04 14:09:20 UTC,1
4,46d53300-e884-4098-be62-a422f9df2988,5844397,499356552,2.38,2019-11-05 17:39:27 UTC,1
...,...,...,...,...,...,...
13324,f58abddb-b147-4ea5-8e2f-5a6ff6e2f9f4,5908190,570033309,1.43,2019-11-29 14:23:17 UTC,1
13325,9f729b71-3877-9415-6570-6a0e81d99441,5908190,542768063,1.43,2019-11-29 16:01:27 UTC,1
13326,74d0f474-f9cd-4e36-932e-b3fcf73595a6,5856537,503269572,9.52,2019-11-28 07:06:15 UTC,1
13327,0d2fc105-b32e-4d75-90a1-ee44d1d0d39d,5909238,579137281,2.62,2019-11-30 17:49:06 UTC,1


array([nan, 'apparel.glove', 'appliances.personal.hair_cutter',
       'appliances.environment.vacuum', 'stationery.cartrige',
       'accessories.bag', 'furniture.bathroom.bath',
       'appliances.environment.air_conditioner'], dtype=object)

In [309]:
pd.DataFrame({'product_id':df_products.product_id.unique()})

,product_id
0,5844397
1,5807857
2,26765
3,5573498
4,5864578
...,...
1995,5909245
1996,5650353
1997,5700958
1998,5833078


In [201]:
sql = """
SELECT *
FROM 'dw.actions'
"""
df = pd.read_sql(sql, conn)
df

,user_id,sku_id,time,model_id,type
0,266079,138778,2016-01-31 23:59:02,NaN,1
1,266079,138778,2016-01-31 23:59:03,0.0,6
2,266079,138778,2016-01-31 23:59:40,0.0,6
3,276956,138778,2016-02-01 01:44:12,217.0,6
4,276956,138778,2016-02-01 01:44:12,223.0,6
...,...,...,...,...,...
13015598,253799,156490,2016-04-15 23:46:06,NaN,1
13015599,253799,156490,2016-04-15 23:46:06,26.0,6
13015600,253799,156490,2016-04-15 23:46:37,26.0,6
13015601,253799,156490,2016-04-15 23:48:42,26.0,6


In [271]:
create_dw_tables = project_dir + r'/src/create_dw_tables.py'

In [272]:
create_dw_tables

'/mnt/data-ubuntu/Projects/user_profile/src/create_dw_tables.py'

In [280]:
%run -i create_dw_tables

ERROR:root:File `'create_dw_tables.py'` not found.
